<a href="https://colab.research.google.com/github/Chygos/UniTartu_Intro_to_DS_Project/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__Research Questions__
- What's the hourly consumption rate for the household?
- What time of the day is electricity consumption the highest or least
- How does electricity consumption vary in the hours, days, or months of the year?
- What is the relationship between electricity consumption and cost?
- Does consumption depend on the atmospheric temperature at that hour?

__To Do Tasks__
- By this week, I will start to do the data visualisation for the research questions we already have. ...more will be added after the modelling part
- Also, @Harry, can you start with the machine learning aspect of it? Since this is a time series task, can you get online resources on how to tackle timeseries analyses such as this? For example, the creation of lag features, which are basically values of past events. 
- I will start with the traditional time series modelling, ARIMA. So, when I am done, I can join you for the machine learning aspect.

Thanks

Chigozie

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
# changing to working directory
import os

os.chdir('gdrive/My Drive/IntroDS_unitartu')

In [3]:
# importing packages 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [4]:
# import train and test sets
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [5]:
# number of rows and columns
train.shape, test.shape

((8592, 13), (168, 12))

In [6]:
train.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
0,2021-09-01 00:00:00+03:00,11.2,10.3,94.0,NaN,NaN,320.0,7.2,16.7,1012.6,2.0,0.09016,0.577
1,2021-09-01 01:00:00+03:00,10.7,9.6,93.0,NaN,NaN,320.0,7.2,13.0,1012.6,2.0,0.09251,0.594
2,2021-09-01 02:00:00+03:00,9.9,9.0,94.0,NaN,NaN,320.0,7.2,13.0,1012.2,2.0,0.08890,0.685
3,2021-09-01 03:00:00+03:00,10.0,8.4,90.0,NaN,NaN,330.0,7.2,13.0,1011.9,1.0,0.08735,1.016
4,2021-09-01 04:00:00+03:00,9.0,8.1,94.0,NaN,NaN,300.0,3.6,13.0,1011.4,2.0,0.08688,0.677


In [7]:
test.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price
0,2022-08-25 00:00:00+03:00,21.1,15.2,69.0,0.0,NaN,340.0,9.0,9.3,1022.0,2.0,0.25533
1,2022-08-25 01:00:00+03:00,20.1,15.1,73.0,0.0,NaN,30.0,6.0,14.8,1022.0,2.0,0.19492
2,2022-08-25 02:00:00+03:00,20.1,15.1,73.0,0.0,NaN,320.0,7.0,13.0,1022.0,2.0,0.18853
3,2022-08-25 03:00:00+03:00,18.7,17.0,90.0,0.0,NaN,0.0,4.0,11.1,1022.4,4.0,0.19947
4,2022-08-25 04:00:00+03:00,18.1,17.1,94.0,0.0,NaN,280.0,7.0,11.1,1022.0,3.0,0.21192


In [8]:
# short description of the train data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8592 entries, 0 to 8591
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         8592 non-null   object 
 1   temp         8592 non-null   float64
 2   dwpt         8592 non-null   float64
 3   rhum         8592 non-null   float64
 4   prcp         2159 non-null   float64
 5   snow         119 non-null    float64
 6   wdir         8592 non-null   float64
 7   wspd         8592 non-null   float64
 8   wpgt         8592 non-null   float64
 9   pres         8592 non-null   float64
 10  coco         8396 non-null   float64
 11  el_price     8592 non-null   float64
 12  consumption  8590 non-null   float64
dtypes: float64(12), object(1)
memory usage: 872.8+ KB


In [9]:
# summary statistics of datasets
train.describe()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
count,8592.000000,8592.000000,8592.000000,2159.000000,119.000000,8592.000000,8592.000000,8592.000000,8592.000000,8396.000000,8592.000000,8590.000000
mean,6.744204,2.486767,77.013617,0.056647,78.319328,201.564246,9.156355,20.869681,1013.229423,4.902930,0.160844,1.046364
std,9.257806,8.184391,17.520566,0.384586,63.129130,87.792064,4.826976,9.956558,12.592944,4.958744,0.120034,1.095114
min,-26.100000,-28.700000,20.000000,0.000000,0.000000,0.000000,0.000000,2.900000,962.600000,1.000000,0.000070,0.000000
25%,0.400000,-2.900000,66.000000,0.000000,20.000000,150.000000,7.200000,13.000000,1006.500000,2.000000,0.092820,0.363000
50%,6.200000,1.900000,83.000000,0.000000,60.000000,210.000000,7.200000,18.500000,1014.700000,3.000000,0.136440,0.811000
75%,13.225000,9.000000,91.000000,0.000000,130.000000,270.000000,10.800000,27.800000,1020.700000,5.000000,0.199845,1.366000
max,31.400000,20.900000,100.000000,7.900000,220.000000,360.000000,31.700000,63.000000,1047.500000,25.000000,4.000000,10.381000


In [10]:
test.describe()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price
count,168.000000,168.000000,168.000000,168.000000,0.0,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,18.204762,13.813095,78.035714,0.283333,NaN,166.220238,8.942262,14.645833,1016.230952,4.851190,0.360046
std,5.152638,3.529424,16.561586,1.138423,NaN,121.531368,5.337378,6.951413,4.087061,5.770708,0.228738
min,8.100000,5.000000,27.000000,0.000000,NaN,0.000000,0.000000,5.500000,1010.000000,1.000000,0.006950
25%,15.050000,13.100000,69.000000,0.000000,NaN,70.000000,5.125000,9.300000,1012.000000,1.000000,0.190323
50%,17.700000,14.700000,83.000000,0.000000,NaN,120.000000,7.000000,13.000000,1016.000000,3.000000,0.298370
75%,21.100000,16.100000,90.500000,0.000000,NaN,310.000000,11.000000,18.500000,1019.050000,5.000000,0.562227
max,30.000000,19.100000,100.000000,6.700000,NaN,360.000000,26.000000,33.300000,1023.200000,25.000000,0.799970


In [11]:
# converting time to date type
train = train.assign(time = pd.to_datetime(train['time'], utc=True).dt.tz_convert('Etc/GMT-3'))
test = test.assign(time = pd.to_datetime(test['time'], utc=True).dt.tz_convert('Etc/GMT-3'))

In [12]:
# check if the time variable is converted to datetype
print(train.dtypes[0])
print(test.dtypes[0])

datetime64[ns, Etc/GMT-3]
datetime64[ns, Etc/GMT-3]


In [13]:
train[train.consumption.isna()]

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
1444,2021-10-31 04:00:00+03:00,7.1,5.2,88.0,NaN,NaN,180.0,10.8,18.5,1015.2,1.0,0.06003,NaN
4972,2022-03-27 04:00:00+03:00,-2.7,-10.5,55.0,NaN,NaN,330.0,14.4,38.9,1020.6,2.0,0.10007,NaN


In [14]:
train[train.snow.isna()]

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
0,2021-09-01 00:00:00+03:00,11.2,10.3,94.0,NaN,NaN,320.0,7.2,16.7,1012.6,2.0,0.09016,0.577
1,2021-09-01 01:00:00+03:00,10.7,9.6,93.0,NaN,NaN,320.0,7.2,13.0,1012.6,2.0,0.09251,0.594
2,2021-09-01 02:00:00+03:00,9.9,9.0,94.0,NaN,NaN,320.0,7.2,13.0,1012.2,2.0,0.08890,0.685
3,2021-09-01 03:00:00+03:00,10.0,8.4,90.0,NaN,NaN,330.0,7.2,13.0,1011.9,1.0,0.08735,1.016
4,2021-09-01 04:00:00+03:00,9.0,8.1,94.0,NaN,NaN,300.0,3.6,13.0,1011.4,2.0,0.08688,0.677
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8587,2022-08-24 19:00:00+03:00,27.1,17.0,54.0,0.0,NaN,180.0,4.0,11.1,1020.0,2.0,0.53494,0.678
8588,2022-08-24 20:00:00+03:00,25.1,17.1,61.0,0.0,NaN,220.0,6.0,11.1,1021.0,1.0,0.49990,0.457
8589,2022-08-24 21:00:00+03:00,24.9,18.1,66.0,0.0,NaN,150.0,4.0,9.3,1020.9,4.0,0.43149,0.500
8590,2022-08-24 22:00:00+03:00,22.1,17.0,73.0,0.0,NaN,160.0,6.0,9.3,1021.0,2.0,0.55203,2.321


In [15]:
train.drop(['prcp', 'snow'], axis=1).corr()

,temp,dwpt,rhum,wdir,wspd,wpgt,pres,coco,el_price,consumption
temp,1.000000,0.901900,-0.412920,-0.041471,0.024502,0.010002,0.110615,-0.289624,0.215842,-0.267864
dwpt,0.901900,1.000000,0.014722,-0.058638,-0.052582,-0.049764,-0.024344,-0.157961,0.192066,-0.250484
rhum,-0.412920,0.014722,1.000000,-0.032963,-0.175986,-0.142185,-0.327359,0.355045,-0.092449,0.094408
wdir,-0.041471,-0.058638,-0.032963,1.000000,0.186036,0.207476,-0.092380,0.073954,-0.067502,-0.005009
wspd,0.024502,-0.052582,-0.175986,0.186036,1.000000,0.804560,-0.333279,0.174252,-0.162252,0.048840
wpgt,0.010002,-0.049764,-0.142185,0.207476,0.804560,1.000000,-0.431564,0.220528,-0.226209,0.072206
pres,0.110615,-0.024344,-0.327359,-0.092380,-0.333279,-0.431564,1.000000,-0.452352,0.157811,-0.068940
coco,-0.289624,-0.157961,0.355045,0.073954,0.174252,0.220528,-0.452352,1.000000,-0.034340,0.133539
el_price,0.215842,0.192066,-0.092449,-0.067502,-0.162252,-0.226209,0.157811,-0.034340,1.000000,-0.124740
consumption,-0.267864,-0.250484,0.094408,-0.005009,0.048840,0.072206,-0.068940,0.133539,-0.124740,1.000000


In [16]:
# filling in precipitation and snow depth data with zeros
train = train.assign(prcp = train.prcp.fillna(0),
                     snow=train.snow.fillna(0))

test = test.assign(prcp = test.prcp.fillna(0),
                   snow=test.snow.fillna(0))


In [17]:
# filling of consumption with past value. The idea is that the consumption for the current hour is the same as the hour preceding it
train = train.assign(consumption = train.consumption.fillna(method='ffill'),
                     coco = train.coco.fillna(-1))

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8592 entries, 0 to 8591
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype                    
---  ------       --------------  -----                    
 0   time         8592 non-null   datetime64[ns, Etc/GMT-3]
 1   temp         8592 non-null   float64                  
 2   dwpt         8592 non-null   float64                  
 3   rhum         8592 non-null   float64                  
 4   prcp         8592 non-null   float64                  
 5   snow         8592 non-null   float64                  
 6   wdir         8592 non-null   float64                  
 7   wspd         8592 non-null   float64                  
 8   wpgt         8592 non-null   float64                  
 9   pres         8592 non-null   float64                  
 10  coco         8592 non-null   float64                  
 11  el_price     8592 non-null   float64                  
 12  consumption  8592 non-null   float64            

# Feature Engineering

Do not run this part. Under review!

In [19]:
train = train.assign(temp = 273+train.temp, 
                     dwpt = 273+train.dwpt)

test = test.assign(temp = 273+test.temp, 
                     dwpt = 273+test.dwpt)

In [20]:
# extracting time related features from the time variable

train = train.assign(hour=train['time'].dt.hour, 
                     weekday=train['time'].dt.isocalendar().day,
                     month=train.time.dt.month, 
                     week=train.time.dt.isocalendar().week)


test = test.assign(hour=test['time'].dt.hour, 
                   weekday=test['time'].dt.isocalendar().day,
                   month=test.time.dt.month, 
                   week=test.time.dt.isocalendar().week)

In [21]:
# saving cleaned data
train.to_csv('data/train_cleaned.csv', index=False)
test.to_csv('data/test_cleaned.csv', index=False)